In [52]:
##Code Section for Declaring Imports##

from bs4 import BeautifulSoup # Load the libraries for beautiful soup
import requests
#import urllib2
import time 
import pandas as pd

from collections import defaultdict

# 1. Code Section for Beautiful Soup Headers

In [53]:
##Code Section for Beautiful Soup Headers##

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://fbref.com/en/comps/9/1526/2016-2017-Premier-League-Stats"
pageTree = requests.get(page, headers=headers)
pageSoup_teams = BeautifulSoup(pageTree.content, 'html.parser')

# 2. Code Section to fetch the Teams - Source : Fbref.com

In [54]:
##Code Section to fetch the Teams ##

Player_Teams = []
Player_Teams_upd_mod = []
Player_Teams_upd = []
table = pageSoup_teams.find('table', class_ = 'min_width sortable stats_table')
for link in table.find_all('a'):
        temp = "https://fbref.com" + link.get("href") 
        #temp_17 = "https://fbref.com" + link.get("href") 
        team_name = link.text
        Player_Teams.append([team_name,temp])
        #Player_Teams.append([team_name,temp_16])

#Player_Teams_upd_2016 = []
#Player_Teams_upd_2017 = []

for value in Player_Teams:
    if value[1][-1:-6:-1] == 'statS':
        Player_Teams_upd.append(value)

[Player_Teams_upd_mod.append(x) for x in Player_Teams_upd if x not in Player_Teams_upd_mod];        

#for value in Player_Teams_2017:
#    if value[1][-1:-6:-1] == 'statS':
#        Player_Teams_upd_2017.append(value)

        
#Player_Teams_upd;        

In [55]:
Player_Teams_upd_mod

[['Chelsea', 'https://fbref.com/en/squads/cff3d9bb/2016-2017/Chelsea-Stats'],
 ['Tottenham',
  'https://fbref.com/en/squads/361ca564/2016-2017/Tottenham-Hotspur-Stats'],
 ['Manchester City',
  'https://fbref.com/en/squads/b8fd03ef/2016-2017/Manchester-City-Stats'],
 ['Liverpool',
  'https://fbref.com/en/squads/822bd0ba/2016-2017/Liverpool-Stats'],
 ['Arsenal', 'https://fbref.com/en/squads/18bb7c10/2016-2017/Arsenal-Stats'],
 ['Manchester Utd',
  'https://fbref.com/en/squads/19538871/2016-2017/Manchester-United-Stats'],
 ['Everton', 'https://fbref.com/en/squads/d3fd31cc/2016-2017/Everton-Stats'],
 ['Southampton',
  'https://fbref.com/en/squads/33c895d4/2016-2017/Southampton-Stats'],
 ['Bournemouth',
  'https://fbref.com/en/squads/4ba7cbea/2016-2017/Bournemouth-Stats'],
 ['West Brom',
  'https://fbref.com/en/squads/60c6b05f/2016-2017/West-Bromwich-Albion-Stats'],
 ['West Ham',
  'https://fbref.com/en/squads/7c21e445/2016-2017/West-Ham-United-Stats'],
 ['Leicester City',
  'https://fbref.

### 2.1 Code Section to fetch the Players from Teams - Source : Fbref.com

In [56]:
##Code Section to fetch Player List from Teams ##

Players_Info = {}
for player_page in Player_Teams_upd:
    page = player_page[1]
    player_team = player_page[0]
    pageTree = requests.get(page, headers=headers)
    pageSoup_teams = BeautifulSoup(pageTree.content, 'html.parser')
    
    for table in pageSoup_teams.find_all('div', id = 'div_stats_standard_1526'):
        rows = [row for row in table.find_all('tr')]
    for index in range(2,len(rows)-2):
        player_name =     rows[index].find_all('th')[0].find('a').text
        player_url  =     'https://fbref.com' + rows[index].find_all('th')[0].find('a')['href']
        player_details =  rows[index].find_all('td')
        Players_Info[player_name] = [player_name] + [player_url] + [player_team] + [i.text for i in player_details[:-1]] 

Players_Info_df = pd.DataFrame(Players_Info).T  #transpose
Players_Info_df.columns = ['Player','PlLink','Team','Nation', 'Pos','Age','MP','Starts','Min','90s','Gls','Ast', 
                    'PK','PKatt','CrdY','CrdR','PGls','PAst','PG+A','PG-PK','PG+A-PK'] 
Players_Info_df.set_index('Player', inplace= True)
Players_Info_df;        

In [57]:
Players_Info_df

,PlLink,Team,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,PK,PKatt,CrdY,CrdR,PGls,PAst,PG+A,PG-PK,PG+A-PK
Player,,,,,,,,,,,,,,,,,,,,
César Azpilicueta,https://fbref.com/en/players/53cad200/Cesar-Az...,Chelsea,es ESP,DF,26,38,38,"3,420",38.0,1,4,0,0,4,0,0.03,0.11,0.13,0.03,0.13
Gary Cahill,https://fbref.com/en/players/7914b9fe/Gary-Cahill,Chelsea,eng ENG,DF,30,37,36,"3,296",36.6,6,0,0,0,5,0,0.16,0.00,0.16,0.16,0.16
Thibaut Courtois,https://fbref.com/en/players/1840e36d/Thibaut-...,Chelsea,be BEL,GK,24,36,36,"3,240",36.0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00
Eden Hazard,https://fbref.com/en/players/a39bb753/Eden-Hazard,Chelsea,be BEL,"FW,MF",25,36,36,"2,982",33.1,16,5,2,3,3,0,0.48,0.15,0.63,0.42,0.57
N'Golo Kanté,https://fbref.com/en/players/b9fbae28/NGolo-Kante,Chelsea,fr FRA,MF,25,35,35,"3,138",34.9,1,1,0,0,9,0,0.03,0.03,0.06,0.03,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mika,https://fbref.com/en/players/d74a54d1/Mika,Sunderland,pt POR,GK,25,0,0,,,,,,,,,,,,,
Ethan Robson,https://fbref.com/en/players/961a5164/Ethan-Ro...,Sunderland,eng ENG,MF,19,0,0,,,,,,,,,,,,,
Josh Robson,https://fbref.com/en/players/e10308d3/Josh-Robson,Sunderland,eng ENG,FW,18,0,0,,,,,,,,,,,,,


### 2.2 Code Section to fetch Player level Attributes - Source : Fbref.com

In [60]:
##Code Section to fetch Specific Player Attributes##
Player_Attr = {}
Player_Shooting = {}
for index, row in enumerate(Players_Info_df.itertuples()):
    page = row.PlLink
    pageTree = requests.get(page, headers=headers)
    pageSoup_player = BeautifulSoup(pageTree.content, 'html.parser')

    #pageSoup_player 
    for table in pageSoup_player.find_all('div', class_ = 'players', id = 'info'):
        #print(table)
        rows = [row for row in table.find_all('p')]
        headings = [heading for heading in table.find_all('h1') ]
    try:   
        #print(headings[0])
        player_name   = headings[0].find('span').text
        #print(player_name)
        player_height = rows[2].find_all('span')[0].text
        player_weight = rows[2].find_all('span')[1].text
        player_footedness = rows[1].text
        Player_Attr[player_name] = [player_name] + [player_height] + [player_weight] + [player_footedness]
    except:
        player_name   = 'Error'
        player_height = 'Error'
        player_weight = 'Error'
        player_footedness = 'Error'
        Player_Attr[player_name] = [player_name] + [player_height] + [player_weight] + [player_footedness]
        continue
#    for table in pageSoup_player.find_all('div', id = 'div_stats_shooting_dom_lg'):
#        rows = [row for row in table.find_all('tr')]
        #for index in range(4:18): 
#        items = rows[12].find_all('td')
#        Player_Shooting[player_name] = [player_name] + [i.text for i in items[4:]]

#print(Player_Shooting)
#Players_Shooting_df = pd.DataFrame(Player_Shooting).T 
#Players_Shooting_df.columns = ['Player','','Weight','Footedness']#print(rows[3])

Players_Attr_df = pd.DataFrame(Player_Attr).T  #transpose
Players_Attr_df.columns = ['Player','Height','Weight','Footedness'] 
Players_Attr_df.set_index('Player', inplace= True)
## Merge the above different Player specific information into a single DataFrame for players
Players_Info_outer_merged = Players_Info_df.merge(Players_Attr_df, how='inner', left_on='Player', right_on='Player')
#Players_Info_outer_merged = pd.merge(Players_Info_df, Players_Attr_df,
#                        how="outer", on=["Player"])

In [61]:
Players_Info_outer_merged

,PlLink,Team,Nation,Pos,Age,MP,Starts,Min,90s,Gls,...,CrdY,CrdR,PGls,PAst,PG+A,PG-PK,PG+A-PK,Height,Weight,Footedness
Player,,,,,,,,,,,,,,,,,,,,,
César Azpilicueta,https://fbref.com/en/players/53cad200/Cesar-Az...,Chelsea,es ESP,DF,26,38,38,"3,420",38.0,1,...,4,0,0.03,0.11,0.13,0.03,0.13,178cm,76kg,"Position: DF (FB, right) ▪ Footed: 92% Right*"
Gary Cahill,https://fbref.com/en/players/7914b9fe/Gary-Cahill,Chelsea,eng ENG,DF,30,37,36,"3,296",36.6,6,...,5,0,0.16,0.00,0.16,0.16,0.16,193cm,86kg,Position: DF (CB) ▪ Footed: 89% Right*
Thibaut Courtois,https://fbref.com/en/players/1840e36d/Thibaut-...,Chelsea,be BEL,GK,24,36,36,"3,240",36.0,0,...,1,0,0.00,0.00,0.00,0.00,0.00,199cm,91kg,Position: GK ▪ Footed: 70% Left*
Eden Hazard,https://fbref.com/en/players/a39bb753/Eden-Hazard,Chelsea,be BEL,"FW,MF",25,36,36,"2,982",33.1,16,...,3,0,0.48,0.15,0.63,0.42,0.57,173cm,75kg,"Position: FW-MF (AM-WM, left) ▪ Footed: 86% R..."
N'Golo Kanté,https://fbref.com/en/players/b9fbae28/NGolo-Kante,Chelsea,fr FRA,MF,25,35,35,"3,138",34.9,1,...,9,0,0.03,0.03,0.06,0.03,0.06,"\n March 29, 1991\n","\n \n in Paris, France\n \n","169cm, 68kg (5-6½, 150lb)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Joleon Lescott,https://fbref.com/en/players/62393d3b/Joleon-L...,Sunderland,eng ENG,DF,33,2,1,124,1.4,0,...,0,0,0.00,0.00,0.00,0.00,0.00,188cm,82kg,Position: DF (CB)
Jeremain Lens,https://fbref.com/en/players/6f1721f0/Jeremain...,Sunderland,nl NED,"FW,MF",28,2,0,69,0.8,0,...,0,0,0.00,0.00,0.00,0.00,0.00,178cm,80kg,"Position: FW-MF (AM-WM, right) ▪ Footed: 81% ..."
Joel Asoro,https://fbref.com/en/players/13e827f8/Joel-Asoro,Sunderland,se SWE,FW,17,1,0,10,0.1,0,...,0,0,0.00,0.00,0.00,0.00,0.00,"\n April 27, 1999\n",\n \n in Sweden\n \n,"176cm, 70kg (5-9, 154lb)"


In [62]:
#Players_Attr_df;

In [63]:
#Players_Info_df_Chelsea = Players_Info_outer_merged[Players_Info_outer_merged.Team == 'Chelsea'];

In [64]:
#Players_Info_df_Chelsea;

# 3. Code Section to fetch markvet value of the Players - Source : Transfermarkt.com

### 3.1 Beautiful Soup Header for Transfermark.com website(for market valuations)

In [65]:
##Code Section to fetch Player Market Valuations from TransferMarket Website##
page = "https://www.transfermarkt.us/premier-league/daten/wettbewerb/GB1"
pageTree = requests.get(page, headers=headers)
pageSoup_teams = BeautifulSoup(pageTree.content, 'html.parser')


### 3.2 Fetch the teams

In [66]:
Teams = []
Teams_mod = []
for table in pageSoup_teams.find_all('table', class_='items'):
    #time.sleep(20)
    for link in table.find_all('a'):
        temp = "https://www.transfermarkt.us" + link.get("href") 
        Teams.append(temp)
  ## Notice that there are redundant links in the list so I am filtering out the duplicate values      
    [Teams_mod.append(x) for x in Teams if x not in Teams_mod]

Teams_corr = Teams_mod[4::2] #This is to clean-up redundant and un-necessary links in the data     
Teams_corr;

In [67]:
Teams_corr[0:5]

['https://www.transfermarkt.us/fc-liverpool/startseite/verein/31',
 'https://www.transfermarkt.us/manchester-city/startseite/verein/281',
 'https://www.transfermarkt.us/fc-chelsea/startseite/verein/631',
 'https://www.transfermarkt.us/manchester-united/startseite/verein/985',
 'https://www.transfermarkt.us/tottenham-hotspur/startseite/verein/148']

In [68]:
team_page_temp = []
team_page_temp = Teams_corr[0:5]
print(team_page_temp)

['https://www.transfermarkt.us/fc-liverpool/startseite/verein/31', 'https://www.transfermarkt.us/manchester-city/startseite/verein/281', 'https://www.transfermarkt.us/fc-chelsea/startseite/verein/631', 'https://www.transfermarkt.us/manchester-united/startseite/verein/985', 'https://www.transfermarkt.us/tottenham-hotspur/startseite/verein/148']


In [75]:
Teams_corr

['https://www.transfermarkt.us/fc-liverpool/startseite/verein/31',
 'https://www.transfermarkt.us/manchester-city/startseite/verein/281',
 'https://www.transfermarkt.us/fc-chelsea/startseite/verein/631',
 'https://www.transfermarkt.us/manchester-united/startseite/verein/985',
 'https://www.transfermarkt.us/tottenham-hotspur/startseite/verein/148',
 'https://www.transfermarkt.us/fc-arsenal/startseite/verein/11',
 'https://www.transfermarkt.us/fc-everton/startseite/verein/29',
 'https://www.transfermarkt.us/leicester-city/startseite/verein/1003',
 'https://www.transfermarkt.us/wolverhampton-wanderers/startseite/verein/543',
 'https://www.transfermarkt.us/aston-villa/startseite/verein/405',
 'https://www.transfermarkt.us/newcastle-united/startseite/verein/762',
 'https://www.transfermarkt.us/west-ham-united/startseite/verein/379',
 'https://www.transfermarkt.us/brighton-amp-hove-albion/startseite/verein/1237',
 'https://www.transfermarkt.us/fc-southampton/startseite/verein/180',
 'https:/

### 3.3 Fetch players from each of the teams

In [203]:
Teams_Players_Names = []
Teams_Players_Valuations_2016 = []
Teams_Players_Valuations_2017 = []
Teams_Players_Names_2017 = []
Teams_Players_Positions = []
Teams_Players = []
Teams_Players_mod = []
team_page_temp = []
Player_2016 = {}
Player_2017 = []
#team_page_temp = Teams_corr[0:11]
player_valuations_dataframe = pd.DataFrame()
#for team_pa'https://www.transfermarkt.us/chelsea-fc/startseite/verein/631?saison_id=20ge in Teams_corr: ## For each team in the list -> we go ahead to fetch the players
test_page = "https://www.transfermarkt.us/fc-everton/startseite/verein/29"
#team_page_temp.append(Teams_corr[0:1])
#team_page_temp.append(test_page)
for team_page in Teams_corr: ## We Look through the list and fetch every team and load their respective pages 
    
    #time.sleep(20)
    #print(team_page)
    #a_dict = collections.defaultdict(list)
    #a_dict["a"].append("hello")
    #print(a_dict)
    #OUTPUT
    #defaultdict(<class 'list'>, {'a': ['hello']})
    #a_dict["a"].append("kite")
    #print(a_dict)

    team_page_2016 = team_page +  "?saison_id=2016"
    pageTree = requests.get(team_page_2016, headers=headers)
    pageSoup_player = BeautifulSoup(pageTree.content, 'html.parser')
    for table in pageSoup_player.find_all('table', class_='items'):
        #for link in table.find_all('td'):
        for player_link in table.find_all('a',class_ = 'spielprofil_tooltip'):
                player_name = player_link.get("title")
                if player_name not in Teams_Players_Names:
                    Teams_Players_Names.append(player_name)
                #print(player_name)
        for player_value_link in table.find_all('td',class_= 'rechts hauptlink'):
                #player_valuation = player_value_link.text
                #if player_valuation == '&nbsp':
                #    print('&nbsp')
                player_valuation = player_value_link.text.split('\xa0')[0]
                if not player_valuation :
                    player_valuation = '0'
                Teams_Players_Valuations_2016.append(player_valuation)
                #player_valuation = player_value_link.text
                #player_valuation = player_valuation.split("\")[0]
                #print(player_valuation)                                                
                #if not player_valuation:
                #    player_valuation = 'N/A'
                #Teams_Players_Valuations_2016.append(player_valuation)
        
                #print(player_valuation)
            #if player_name not in Player_2016:
    Teams_Players_Valuations_2016 = Teams_Players_Valuations_2016[0:776]
    Player_2016 = {'Player': Teams_Players_Names, 'Valuation-2016':Teams_Players_Valuations_2016 }    
    
    team_page_2017 = team_page +  "?saison_id=2017"
    pageTree = requests.get(team_page_2017, headers=headers)
    pageSoup_player = BeautifulSoup(pageTree.content, 'html.parser')
    for table in pageSoup_player.find_all('table', class_='items'):
        #for link in table.find_all('td'):
        for player_link in table.find_all('a',class_ = 'spielprofil_tooltip'):
                player_name = player_link.get("title")
                if player_name not in Teams_Players_Names_2017:
                    Teams_Players_Names_2017.append(player_name)
                #print(player_name)
        for player_value_link in table.find_all('td',class_= 'rechts hauptlink'):
                #player_valuation = player_value_link.text
                #if player_valuation == '&nbsp':
                #    print('&nbsp')
                player_valuation = player_value_link.text.split('\xa0')[0]
                if not player_valuation :
                    player_valuation = '0'
                Teams_Players_Valuations_2017.append(player_valuation)
                #player_valuation = player_value_link.text
                #player_valuation = player_valuation.split("\")[0]
                #print(player_valuation)                                                
                #if not player_valuation:
                #    player_valuation = 'N/A'
                #Teams_Players_Valuations_2016.append(player_valuation)
        
                #print(player_valuation)
            #if player_name not in Player_2016:
            
    Teams_Players_Valuations_2016 = Teams_Players_Valuations_2016[0:776]
    Player_2016 = {'Player': Teams_Players_Names, 'Valuation-2016':Teams_Players_Valuations_2016 }    
    Teams_Players_Valuations_2017 = Teams_Players_Valuations_2017[0:732]
    Player_2017 = {'Player': Teams_Players_Names_2017, 'Valuation-2017':Teams_Players_Valuations_2017 }    
    
    
    #team_page_2017 = team_page +  "?saison_id=2017"
    #pageTree = requests.get(team_page_2017, headers=headers)
    #pageSoup_player = BeautifulSoup(pageTree.content, 'html.parser')
    #for table in pageSoup_player.find_all('table', class_='items'):
        #rows = [row for row in table.find_all('a',class_ = 'spielprofil_tooltip')]
        #headings = [heading for heading in table.find_all('h1') ]
    #try:   
        #print(headings[0])
    #    player_name   = headings[0].find('span').text
        #print(player_name)
    #    player_height = rows[2].find_all('span')[0].text
    #    player_weight = rows[2].find_all('span')[1].text
    #    player_footedness = rows[1].text
    #    Player_Attr[player_name] = [player_name] + [player_height] + [player_weight] + [player_footedness]
  
    #    for link in table.find_all('td'):
    #        for link in table.find_all('a',class_ = 'spielprofil_tooltip'):
     #           player_name = link.get("title")
     #           print(player_name)
     #       for link in table.find_all('td',class_= 'rechts hauptlink'):
      #          player_valuation = link.text
      #          print(player_valuation)
            #if player_name not in Player_2017:
       #     Player_2017.append([player_name,player_valuation])
                
        #for link in table.find_all('a', class_ = 'spielprofil_tooltip'):
            #print(link)
        #    if (link.get("title")) not in Player_2016:
         #       Player_2016[link.get("title")] = link.get("title")
            #else:
                #Player_2016.setdefault(link.get("title"), [])
                #Player_2016[link.get("title")].append(link.get("title"))
                #Teams_Players_Names.append(link.get("title"))
                #Player_Attr[link.get("title")] = link.get("title")

        #for link in table.find_all('td', class_ = 'rechts hauptlink'):
                #Player_2016.setdefault(link.get("title"), [])
        #        print(link)
        #        print(link.get("title"))
        #        print(link.text)
        #        Player_2016[link.get("title")] = link.text
                #Teams_Players_Valuations_2016.append(link.text)
    
    #team_page = team_page +  "?saison_id=2017"
    #pageTree = requests.get(team_page, headers=headers)
    #pageSoup_player = BeautifulSoup(pageTree.content, 'html.parser')
    #for table in pageSoup_player.find_all('table', class_='items'):
    #    for link in table.find_all('a', class_ = 'spielprofil_tooltip'):
    #        if (link.get("title")) not in Player_2017:
    #            Player_2017[link.get("title")] = link.get("title")
        #for link in table.find_all('a', class_ = 'spielprofil_tooltip'):
        #    if (link.get("title")) not in Teams_Players_Names:
        #        Teams_Players_Names.append(link.get("title"))
                #Player_Attr[link.get("title")] = link.get("title")
    #    for link in table.find_all('td', class_ = 'rechts hauptlink'):
    #            Player_2017[link.get("title")] = link.text
                #Teams_Players_Valuations_2017.append(link.text)
                
                
Players_Attr_df_2016 = pd.DataFrame(Player_2016, columns = ['Player','Valuation-2016'])  #transpose
Players_Attr_df_2016.columns = ['Player','Valuation-2016']
Players_Attr_df_2016.set_index('Player', inplace= True)
Players_Attr_df_2017 = pd.DataFrame(Player_2017, columns = ['Player','Valuation-2017'])  #transpose
Players_Attr_df_2017.columns = ['Player','Valuation-2017']
Players_Attr_df_2017.set_index('Player', inplace= True)
merged_valuation_df = Players_Attr_df_2016.merge(Players_Attr_df_2017, how='inner', left_on='Player', right_on='Player')
#Players_Attr_df.columns = ['Player','Valuation_2016','Valuation_2017'] 
#Players_Attr_df.set_index('Player', inplace= True)
#Players_Attr_df['Player'] = Teams_Players_Names
#Players_Attr_df['Valuation_2016'] = Teams_Players_Valuations_2016
#Players_Attr_df['Valuation_2017'] = Teams_Players_Valuations_2017
#Players_Attr_df.set_index('Player', inplace= True)
print(len(Teams_Players_Valuations_2017))
print(len(Teams_Players_Names_2017))

732
732


In [206]:
merged_valuation_df_copy = merged_valuation_df


In [180]:
Teams_Players_Valuations_2016;

In [181]:
Teams_Players_Names;

In [182]:
table;


In [113]:
player_valuation

'$1.65m\xa0\xa0'

In [79]:
team_page_2016


'https://www.transfermarkt.us/west-bromwich-albion/startseite/verein/984?saison_id=2016'

In [184]:
#merged_valuation_df;

### 3.4 Merge with the Final Dataset

In [209]:
Players_Dataset = Players_Info_outer_merged.merge(merged_valuation_df,how='inner', left_on='Player', right_on='Player')

In [211]:
Players_Dataset.head(100)

,PlLink,Team,Nation,Pos,Age,MP,Starts,Min,90s,Gls,...,PGls,PAst,PG+A,PG-PK,PG+A-PK,Height,Weight,Footedness,Valuation-2016,Valuation-2017
Player,,,,,,,,,,,,,,,,,,,,,
César Azpilicueta,https://fbref.com/en/players/53cad200/Cesar-Az...,Chelsea,es ESP,DF,26,38,38,"3,420",38.0,1,...,0.03,0.11,0.13,0.03,0.13,178cm,76kg,"Position: DF (FB, right) ▪ Footed: 92% Right*",$33.00m,$44.00m
Gary Cahill,https://fbref.com/en/players/7914b9fe/Gary-Cahill,Chelsea,eng ENG,DF,30,37,36,"3,296",36.6,6,...,0.16,0.00,0.16,0.16,0.16,193cm,86kg,Position: DF (CB) ▪ Footed: 89% Right*,$17.60m,$16.50m
Thibaut Courtois,https://fbref.com/en/players/1840e36d/Thibaut-...,Chelsea,be BEL,GK,24,36,36,"3,240",36.0,0,...,0.00,0.00,0.00,0.00,0.00,199cm,91kg,Position: GK ▪ Footed: 70% Left*,$44.00m,$66.00m
Eden Hazard,https://fbref.com/en/players/a39bb753/Eden-Hazard,Chelsea,be BEL,"FW,MF",25,36,36,"2,982",33.1,16,...,0.48,0.15,0.63,0.42,0.57,173cm,75kg,"Position: FW-MF (AM-WM, left) ▪ Footed: 86% R...",$82.50m,$121.00m
N'Golo Kanté,https://fbref.com/en/players/b9fbae28/NGolo-Kante,Chelsea,fr FRA,MF,25,35,35,"3,138",34.9,1,...,0.03,0.03,0.06,0.03,0.06,"\n March 29, 1991\n","\n \n in Paris, France\n \n","169cm, 68kg (5-6½, 150lb)",$55.00m,$66.00m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Carl Jenkinson,https://fbref.com/en/players/874125a3/Carl-Jen...,Arsenal,eng ENG,DF,24,1,1,82,0.9,0,...,0.00,0.00,0.00,0.00,0.00,187cm,76kg,"Position: DF (FB, right) ▪ Footed: 85% Right*",$5.50m,$3.85m
Mathieu Debuchy,https://fbref.com/en/players/92232aad/Mathieu-...,Arsenal,fr FRA,DF,31,1,1,15,0.2,0,...,0.00,0.00,0.00,0.00,0.00,"\n July 28, 1985\n","\n \n in Fretin, France\n \n","177cm, 74kg (5-9½, 163lb)",$3.85m,$4.95m
Jack Wilshere,https://fbref.com/en/players/9c318325/Jack-Wil...,Bournemouth,eng ENG,MF,24,27,22,"1,910",21.2,0,...,0.00,0.09,0.09,0.00,0.09,172cm,66kg,Position: MF ▪ Footed: 93% Left*,$19.80m,$44.00m


In [ ]:
Players_Dataset.dropna();

### 3.3 Fetch information from Wikipedia

In [40]:
import pageviewapi
page_views = pageviewapi.per_article('en.wikipedia', 'Rooney', '20151106', '20151120',
                        access='all-access', agent='all-agents', granularity='daily')
#print(page_views['items'])

In [41]:
Players_Attr_df;

In [42]:

Players_Dataset = pd.merge(Players_Attr_df,Players_Info_df_Chelsea,
                        how="outer", on=["Player"])

In [43]:
#Players_Dataset.groupby("Team").head(25);

In [46]:
Players_Dataset.dropna();

# 4. Data Models 

### 4.1 Pick Selective DataSet for Model-1 

In [16]:
Players_Dataset[(Players_Dataset.Team == 'Chelsea')]

,Valuation,PlLink,Team,Nation,Pos,Age,MP,Starts,Min,90s,...,CrdY,CrdR,PGls,PAst,PG+A,PG-PK,PG+A-PK,Height,Weight,Footedness
Player,,,,,,,,,,,,,,,,,,,,,
Thibaut Courtois,$44.00m,https://fbref.com/en/players/1840e36d/Thibaut-...,Chelsea,be BEL,GK,24,36,36,"3,240",36.0,...,1,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
David Luiz,$33.00m,https://fbref.com/en/players/c0e27d1a/David-Luiz,Chelsea,br BRA,DF,29,33,33,"2,954",32.8,...,6,0,0.03,0.00,0.03,0.03,0.03,NaN,NaN,NaN
Gary Cahill,$17.60m,https://fbref.com/en/players/7914b9fe/Gary-Cahill,Chelsea,eng ENG,DF,30,37,36,"3,296",36.6,...,5,0,0.16,0.00,0.16,0.16,0.16,193cm,86kg,Position: DF (CB) ▪ Footed: 89% Right*
Kurt Zouma,$16.50m,https://fbref.com/en/players/ce4246f5/Kurt-Zouma,Chelsea,fr FRA,DF,21,9,3,250,2.8,...,0,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
John Terry,$1.10m,https://fbref.com/en/players/68517cfd/John-Terry,Chelsea,eng ENG,DF,35,9,6,496,5.5,...,1,0,0.18,0.00,0.18,0.18,0.18,NaN,NaN,NaN
Marcos Alonso,$27.50m,https://fbref.com/en/players/f4290206/Marcos-A...,Chelsea,es ESP,MF,25,31,30,"2,693",29.9,...,2,0,0.20,0.10,0.30,0.20,0.30,NaN,NaN,NaN
César Azpilicueta,$33.00m,https://fbref.com/en/players/53cad200/Cesar-Az...,Chelsea,es ESP,DF,26,38,38,"3,420",38.0,...,4,0,0.03,0.11,0.13,0.03,0.13,178cm,76kg,"Position: DF (FB, right) ▪ Footed: 92% Right*"
Ola Aina,$1.10m,https://fbref.com/en/players/246d153b/Ola-Aina,Chelsea,ng NGA,"DF,MF",19,3,0,26,0.3,...,0,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
Nathaniel Chalobah,$5.50m,https://fbref.com/en/players/422862ea/Nathanie...,Chelsea,eng ENG,MF,21,10,1,167,1.9,...,2,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
